In [1]:
import pandas as pd
import nltk
nltk.download('sentiwordnet')
from nltk.corpus import sentiwordnet as swn

[nltk_data] Downloading package sentiwordnet to C:\Users\Luo
[nltk_data]     Junru\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [2]:
df = pd.read_csv('sgtransport1032.csv', header=0, encoding='utf-8')
df['Text'] = df['Text'].str.lower()
df=df[['Sentiment Num','Text']]
df['Text'].dtypes
df['Text'] = df['Text'].astype(str)
print(df.head())

   Sentiment Num                                               Text
0             -1  comfortdelgro investigating after maxicab driv...
1              1  no choice lol...this nasi lemak very nice one ...
2              1  i still cant get over the comfortdelgro at the...
3             -1  urine, stool in our bodies are somewhat uncomf...
4              1  he cannot tahan lah what to do...pee in his pa...


In [3]:
def sentiment_calc(doc):
    try:
        sentences = nltk.sent_tokenize(doc)
        stokens = [nltk.word_tokenize(sent) for sent in sentences]
        taggedlist=[]
        for stoken in stokens:        
             taggedlist.append(nltk.pos_tag(stoken))
        wnl = nltk.WordNetLemmatizer()

        score_list=[]
        for idx,taggedsent in enumerate(taggedlist):
            score_list.append([])
            for idx2,t in enumerate(taggedsent):
                newtag=''
                lemmatized=wnl.lemmatize(t[0])
                if t[1].startswith('NN'):
                    newtag='n'
                elif t[1].startswith('JJ'):
                    newtag='a'
                elif t[1].startswith('V'):
                    newtag='v'
                elif t[1].startswith('R'):
                    newtag='r'
                else:
                    newtag=''       
                if(newtag!=''):    
                    synsets = list(swn.senti_synsets(lemmatized, newtag))
                    #Getting average of all possible sentiments, as you requested        
                    score=0
                    if(len(synsets)>0):
                        for syn in synsets:
                            score+=syn.pos_score()-syn.neg_score()
                        score_list[idx].append(score/len(synsets))
        sentence_sentiment=[]

        def condition(x): 
            return x!=0

        for score_sent in score_list:
            sentence_sentiment.append(sum([word_score for word_score in score_sent])/len(score_sent))
        print("Sentiment for each sentence for:"+doc)
        print(sentence_sentiment)

        return sum([sentence for sentence in sentence_sentiment])/len(sentence_sentiment)
    except:
        return None

df['polarity'] = df['Text'].apply(sentiment_calc)

Sentiment for each sentence for:comfortdelgro investigating after maxicab driver allegedly parked illegally to eat nasi lemak follow @sg_malayalees #singaporemalayalees https://t.co/nsanhpnnlf  https://t.co/seykcned8k
[0.05859375]
Sentiment for each sentence for:no choice lol...this nasi lemak very nice one and also very hungry mah..give chance lah...ðÿ˜‚ comfortdelgro investigating after maxicab driver allegedly parked illegally -- to eat nasi lemak https://t.co/xshudwxnxd
[0.08125]
Sentiment for each sentence for:i still cant get over the comfortdelgro at the back fml lah free promo hahahaha cutie:; https://t.co/b4oarjrytb
[-0.08333333333333334]
Sentiment for each sentence for:urine, stool in our bodies are somewhat uncomfortable where ever we go must be aware where the toilets are situated.ðÿ˜·ðÿ¤’ðÿ˜¬ðÿ™„ðÿ˜¢ðÿ˜³ asiaone: comfortdelgro gives cabby 'severe warning' after he was caught peeing in public. https://t.co/yhwny9ixhy
[-0.025703905768560947, 0.0]
Sentiment for each sentence 

In [4]:
df['new sentiment'] = 0
df.loc[(df['polarity'] > 0.2) & (df['polarity'] <= 1), 'new sentiment'] = 1.0
df.loc[(df['polarity'] < 0), 'new sentiment'] = -1.0
print(df.head(10))

   Sentiment Num                                               Text  polarity  \
0             -1  comfortdelgro investigating after maxicab driv...  0.058594   
1              1  no choice lol...this nasi lemak very nice one ...  0.081250   
2              1  i still cant get over the comfortdelgro at the... -0.083333   
3             -1  urine, stool in our bodies are somewhat uncomf... -0.012852   
4              1  he cannot tahan lah what to do...pee in his pa... -0.080797   
5             -1  unacceptable, frustrating process in handling ... -0.043915   
6             -1  they employ all these foreigners (malaysians) ... -0.007356   
7              1  @adamekeen could do with comfortdelgro giving ...  0.108663   
8              0  comfortdelgro's self-driving shuttles to start... -0.000893   
9              0  @sktoh6 uber has more revenue compared to comf...  0.041667   

   new sentiment  
0            0.0  
1            0.0  
2           -1.0  
3           -1.0  
4           -

In [5]:
from sklearn.metrics import accuracy_score
# df.loc[df['Sentiment'].str.contains('neutral'), 'Sentiment Num'] = 0.0
# df.loc[df['Sentiment'].str.contains('positive'), 'Sentiment Num'] = 1.0
# df.loc[df['Sentiment'].str.contains('negative'), 'Sentiment Num'] = -1.0
accuracy_score(df["Sentiment Num"], df["new sentiment"])

0.5232558139534884

In [6]:
from sklearn.metrics import f1_score

In [7]:
f1_score(df['Sentiment Num'], df['new sentiment'], average='weighted')

0.4913621460153376